In [1]:
# Basic
from collections import Counter
import datetime
import inspect
import math
import numpy as np
import os
import pprint
import sys

# yaml specific
import yaml

# Data handling
from fuzzywuzzy import fuzz, process
import pandas as pd
from tqdm import tqdm

# This is to add the root project folder to sys.path so that imports are easy
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

# my library
from src.db_utils import update_player, add_player 
from src.core import runs_scored
from src.insights import total_runs

C:\Users\aniru\miniconda3\envs\torch\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
# Config variables
raw_data_path = os.path.join("..", "raw_data") 
clean_data_path = os.path.join("..", "clean_data") 

In [5]:
df_tournament = pd.read_csv(os.path.join(clean_data_path, "tournament.csv"))
df_tournament = df_tournament.loc[:, ~df_tournament.columns.str.contains('^Unnamed')]
tournament_id_map = dict(zip(df_tournament.tournament_name, df_tournament.tournament_id))

In [8]:
df_venue = pd.read_csv(os.path.join(clean_data_path, "venue.csv"))
df_venue = df_venue.loc[:, ~df_venue.columns.str.contains('^Unnamed')]
venue_id_map = dict(zip(df_venue.venue_name, df_venue.venue_id))

# Mapping the duplicates as well to its correct venue ids
# IPL maps
venue_id_map["Punjab Cricket Association IS Bindra Stadium, Mohali"] = venue_id_map["Punjab Cricket Association Stadium, Mohali"]
venue_id_map["M.Chinnaswamy Stadium"] = venue_id_map["M Chinnaswamy Stadium"]
# BBL maps
#venue_id_map["Brisbane Cricket Ground"] = venue_id_map["Brisbane Cricket Ground, Woolloongabba"]

In [9]:
df_team = pd.read_csv(os.path.join(clean_data_path, "team.csv"))
df_team = df_team.loc[:, ~df_team.columns.str.contains('^Unnamed')]
team_id_map = dict(zip(df_team.team_name, df_team.team_id))

In [10]:
df_player = pd.read_csv(os.path.join(clean_data_path, "player.csv"))
df_player = df_player.loc[:, ~df_player.columns.str.contains('^Unnamed')]
player_dispname_id_map = dict(zip(df_player.player_display_name, df_player.player_id))
player_name_id_map = dict(zip(df_player.player_name, df_player.player_id))

In [11]:
df_match = pd.read_csv(os.path.join(clean_data_path, "match.csv"))
df_match = df_match.loc[:, ~df_match.columns.str.contains('^Unnamed')]

df_ball = pd.read_csv(os.path.join(clean_data_path, "ball.csv"))
df_ball = df_ball.loc[:, ~df_ball.columns.str.contains('^Unnamed')]

C:\Users\aniru\miniconda3\envs\torch\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
player_name = "Virat Kohli"
player_id = player_name_id_map[player_name]
venue_name = "ALL"
match_format = "T20"
tournaments = []
years_range = [str(year) for year in range(2000, 2022)]
overs_range = [0, 20]
top_n=20

In [16]:
list(set(np.array(df_ball["dismissal_type"])))

[nan,
 'caught',
 'obstructing the field',
 'caught and bowled',
 'bowled',
 'run out',
 'hit wicket',
 'lbw',
 'retired hurt',
 'stumped']